In [1]:
import torch
torch.cuda.empty_cache()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import copy
import torch
from tqdm import tqdm
import pandas as pd
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [ ]:
# 기존 데이터셋 삭제
dataset_name = 'coco_trash_test'
if dataset_name in DatasetCatalog:
    DatasetCatalog.remove(dataset_name)
if dataset_name in MetadataCatalog:
    MetadataCatalog.remove(dataset_name)

# Register Dataset
try:
    register_coco_instances('coco_trash_test', {}, '/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/Split_data/valid_1_10.json', '/data/ephemeral/home/data/dataset/')
except AssertionError:
    pass

In [30]:
from detectron2.config import LazyConfig, instantiate
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import build_detection_test_loader
import torch
import pandas as pd
from tqdm import tqdm
import os
import numpy as np

# mapper - input data를 어떤 형식으로 return할지
def MyMapper(dataset_dict):
    
    # deepcopy를 통해 원본 데이터에 영향을 주지 않음
    dataset_dict = copy.deepcopy(dataset_dict)
    
    # file_name 확인
    if 'file_name' not in dataset_dict:
        raise ValueError("dataset_dict does not contain 'file_name'.")
    
    # 이미지 읽기
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    image_tensor = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))

    # 배치 차원 추가 (1, C, H, W)
    #image_tensor = image_tensor.unsqueeze(0)

    
    # 이미지 데이터 추가
    #image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
    dataset_dict['image'] = image_tensor  # 디버깅용 출력
    # 변환된 데이터셋 반환
    return dataset_dict

# LazyConfig 설정 파일 로드
cfg = LazyConfig.load("/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/swin_rcnn/output/config.yaml")

# 모델 인스턴스 생성
model = instantiate(cfg.model)
model = model.to(torch.device("cuda"))

cfg.MODEL = instantiate(cfg.model)
cfg.DATALOADER = instantiate(cfg.dataloader)
# 제안을 사용하지 않도록 설정
cfg.MODEL.LOAD_PROPOSALS = False
cfg.DATALOADER.test.dataset.names='coco_trash_test'
cfg.DATALOADER.NUM_WORKERS=4,
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS=True

# # num_workers와 batch_size가 ListConfig일 수 있으므로 int로 변환
# cfg.DATALOADER.NUM_WORKERS = int(cfg.DATALOADER.NUM_WORKERS)
# cfg.DATALOADER.BATCH_SIZE = int(cfg.DATALOADER.BATCH_SIZE)
# 체크포인트 로드
checkpointer = DetectionCheckpointer(model)
checkpointer.load("/data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/swin_rcnn/output/model_final.pth")

# 모델을 평가 모드로 전환
model.eval()

# test_loader 설정
test_loader = build_detection_test_loader(cfg,'coco_trash_test',mapper=MyMapper,
                                        num_workers=cfg.DATALOADER.NUM_WORKERS[0])

# 예측 수행
prediction_strings = []
file_names = []

for data in tqdm(test_loader):
    
    prediction_string = ''
    data=data[0]
    with torch.no_grad():
        batched_inputs = [{"image": data['image'].to(torch.device("cuda"))}]
        outputs = model(batched_inputs)[0]['instances']  # model에 올바른 형식으로 전달
    
    # 예측 결과 처리
    targets = outputs.pred_classes.cpu().tolist()
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    scores = outputs.scores.cpu().tolist()
    
    for target, box, score in zip(targets, boxes, scores):
        prediction_string += (str(target) + ' ' + str(score) + ' ' + str(box[0]) + ' ' 
        + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ')
    
    prediction_strings.append(prediction_string)
    file_names.append(data['file_name'].replace('/data/ephemeral/home/data/dataset/train/', ''))

# 제출 파일 생성
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.train.output_dir, 'submission_det2.csv'), index=False)


[10/18 12:31:15 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/swin_rcnn/output/model_final.pth ...
[10/18 12:31:17 d2.data.datasets.coco]: Loaded 550 images in COCO format from /data/ephemeral/home/Seungcheol/level2-objectdetection-cv-07/Split_data/valid_1_10.json
[10/18 12:31:17 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[10/18 12:31:17 d2.data.common]: Serializing 550 elements to byte tensors and concatenating them all ...
[10/18 12:31:17 d2.data.common]: Serialized dataset takes 0.26 MiB


100%|██████████| 550/550 [02:37<00:00,  3.50it/s]


In [34]:
for data in tqdm(test_loader):
    data=data[0]
    with torch.no_grad():
        batched_inputs = [{"image": data['image'].to(torch.device("cuda"))}]
        outputs = model(batched_inputs)[0]['instances']  # model에 올바른 형식으로 전달
    print(outputs.pred_classes)
    print(data['file_name'].replace('/data/ephemeral/home/data/dataset/train/', ''))
    break

  0%|          | 0/550 [00:00<?, ?it/s]

tensor([], device='cuda:0', dtype=torch.int64)
0012.jpg
